In [2]:
import pandas as pd
import numpy as np

In [3]:
ENFJ = pd.read_csv('mbti_enfj.csv', encoding='utf-8')
ENFP = pd.read_csv('mbti_enfp.csv', encoding='utf-8')
ENTJ = pd.read_csv('mbti_entj.csv', encoding='utf-8')
ENTP = pd.read_csv('mbti_entp.csv', encoding='utf-8')
ESFJ = pd.read_csv('mbti_esfj.csv', encoding='utf-8')
ESFP = pd.read_csv('mbti_esfp.csv', encoding='utf-8')
ESTJ = pd.read_csv('mbti_estj.csv', encoding='utf-8')
ESTP = pd.read_csv('mbti_estp.csv', encoding='utf-8')
INFJ = pd.read_csv('mbti_infj.csv', encoding='utf-8')
INFP = pd.read_csv('mbti_infp.csv', encoding='utf-8')
INTJ = pd.read_csv('mbti_intj.csv', encoding='utf-8')
INTP = pd.read_csv('mbti_intp.csv', encoding='utf-8')
ISFJ = pd.read_csv('mbti_isfj.csv', encoding='utf-8')
ISFP = pd.read_csv('mbti_isfp.csv', encoding='utf-8')
ISTJ = pd.read_csv('mbti_istj.csv', encoding='utf-8')
ISTP = pd.read_csv('mbti_istp.csv', encoding='utf-8')

In [4]:
MBTI = {'ENFJ': ENFJ, 'ENFP': ENFP, 'ENTJ': ENFJ, 'ENTP': ENFP, 'ESFJ': ESFJ, 'ESFP': ESFP, 'ESTJ': ESTJ, 'ESTP': ESTP, 'INFJ': INFJ, 'INFP': INFP, 'INTJ': INTJ, 'INTP': INTP, 'ISFJ': ISFJ, 'ISFP': ISFP, 'ISTJ': ISTJ, 'ISTP': ISTP}


In [5]:
for i in MBTI:
    print(i)
    try:
        MBTI.get(i).drop(['Unnamed: 0'], axis=1, inplace=True) # 크롤링 데이터 index 제거
    except:
        print("해당 열 없음")
    MBTI.get(i).dropna(axis=0, inplace=True) # nan 값 있는 행 제거
    MBTI.get(i)['content'] = MBTI.get(i)['content'].str.replace('\n', ' ') # 줄바꿈 제거
    MBTI.get(i).reset_index(drop=True, inplace=True) # 인덱스 초기화

ENFJ
ENFP
ENTJ
해당 열 없음
ENTP
해당 열 없음
ESFJ
ESFP
ESTJ
ESTP
INFJ
INFP
INTJ
INTP
ISFJ
ISFP
ISTJ
ISTP


* * *
# Text Preprocessing

In [7]:
import pykospacing
import kss
import copy
import re
from nltk.tokenize import TreebankWordTokenizer
from konlpy.tag import Komoran
from tqdm import tqdm
import time
import emoji
import json
import requests
import warnings

warnings.filterwarnings('ignore')
tokenizer = TreebankWordTokenizer()
komoran = Komoran()

In [8]:
from hanspell import spell_checker

In [9]:
with open('./stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.readlines()
stopwords = [x.replace('\n','') for x in stopwords]

In [10]:
def lemmatize(tag): # Lemmatize 수행
    if tag[1] == 'VA' or tag[1] == 'VV':
        return tag[0] + '다'
    else:
        return tag[0]

In [11]:
def remove_emoji(text): # 이모지 제거 후 공백으로 대체.
    return emoji.get_emoji_regexp().sub(u'', text)

In [12]:
def hs_additional(text): # 500자 이상의 글에 대해서 맞춤법 검사기 돌리도록 하는 코드
    checked_text = ''
    for i in range(len(text)//500+1):
        try:
            paragraph = text[500*i:500*i+499]
            checked_text += spell_checker.check(paragraph).checked
        except:
            paragraph = text[500*i:]
            checked_text += spell_checker.check(paragraph).checked
    return checked_text

In [13]:
def preprocess_korean_ver2(text):
    try:
        result = []
        document = copy.copy(text)

        # 이모지와 엔터 제거
        document = document.replace('\n', '')
        document = remove_emoji(document)

        # 띄어쓰기 교정 시작
        spacer = pykospacing.Spacing()
        document = spacer(document)

        # 맞춤법 교정
        document = hs_additional(document)

        # 단어 토큰화 & pos tag
        tokens_and_pos = komoran.pos(document)
        p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣0-9A-Za-z ]')

        for token in tokens_and_pos:
            word = p.sub('', token[0])

            if word not in stopwords:
                if token[1] in ['EF','EP','EC','ETM','ETN','JX','JK','SF','SH','SP','SS','SW','JC','JKB','JKC','JKG','JKO','JKQ','JKS','JKV','JX','NA']:
                    continue
                else:
                    if p.sub('', lemmatize(token)) == '':
                        continue
                    else:
                        result.append(p.sub('', lemmatize(token)))

        result = ' '.join(result)

        return result
    
    except:
        print("오류!")

***
# MBTI별로 함수 적용

In [14]:
def preprocess_and_update_progress(row): # 진행 상황 확인하는 tqdm 패키지 활용
    after = preprocess_korean_ver2(row)
    progress_bar.update(1)
    return after

## ENFJ

In [14]:
progress_bar = tqdm(total=len(MBTI['ENFJ']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @")

MBTI['ENFJ']['pp_content'] = MBTI['ENFJ']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   2%|@                                                             | 15/859 [00:34<28:06,  2.00s/it]

오류!


Text Preprocessing:   3%|@@                                                            | 28/859 [01:00<24:32,  1.77s/it]

오류!


Text Preprocessing:  31%|@@@@@@@@@@@@@@@@@@                                           | 262/859 [07:17<16:00,  1.61s/it]

...

Text Preprocessing:  86%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@         | 742/859 [21:49<01:51,  1.05it/s]

오류!


Text Preprocessing:  88%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@        | 753/859 [22:14<03:04,  1.74s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 859/859 [26:19<00:00,  1.84s/it]


In [15]:
MBTI['ENFJ'].to_csv('MBTI_ENFJ_after.csv', encoding='utf-8', index=False)

## ENFP

In [21]:
progress_bar = tqdm(total=len(MBTI['ENFP']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" ●", colour='blue')

MBTI['ENFP']['pp_content'] = MBTI['ENFP']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   0%|                                                               | 4/865 [00:08<27:18,  1.90s/it]

오류!


Text Preprocessing:  15%|●●●●●●●●                                                     | 126/865 [03:52<18:59,  1.54s/it]

오류!


Text Preprocessing:  18%|●●●●●●●●●●                                                   | 152/865 [04:36<24:24,  2.05s/it]

...

Text Preprocessing:  86%|●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●         | 741/865 [21:18<02:20,  1.13s/it]

오류!


Text Preprocessing:  97%|●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●  | 842/865 [24:07<00:31,  1.39s/it]

오류!


Text Preprocessing: 100%|●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●●| 865/865 [24:42<00:00,  1.71s/it]


In [22]:
MBTI['ENFP'].to_csv('MBTI_ENFP_after.csv', encoding='utf-8', index=False)

## ENTJ

In [23]:
progress_bar = tqdm(total=len(MBTI['ENTJ']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" ■", colour='blue')

MBTI['ENTJ']['pp_content'] = MBTI['ENTJ']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   2%|■                                                             | 15/859 [00:32<27:10,  1.93s/it]

오류!


Text Preprocessing:   3%|■■                                                            | 28/859 [00:58<24:52,  1.80s/it]

오류!


Text Preprocessing:  31%|■■■■■■■■■■■■■■■■■■                                           | 262/859 [07:17<15:38,  1.57s/it]

...

Text Preprocessing:  86%|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■         | 742/859 [22:03<02:04,  1.07s/it]

오류!


Text Preprocessing:  88%|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■        | 753/859 [22:31<03:25,  1.93s/it]

오류!


Text Preprocessing: 100%|■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■| 859/859 [27:05<00:00,  1.89s/it]


In [24]:
MBTI['ENTJ'].to_csv('MBTI_ENTJ_after.csv', encoding='utf-8', index=False)

## ENTP

In [25]:
progress_bar = tqdm(total=len(MBTI['ENTP']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @", colour='red')

MBTI['ENTP']['pp_content'] = MBTI['ENTP']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   0%|                                                               | 4/865 [00:09<30:21,  2.12s/it]

오류!


Text Preprocessing:  15%|@@@@@@@@                                                     | 126/865 [04:21<20:30,  1.67s/it]

오류!


Text Preprocessing:  18%|@@@@@@@@@@                                                   | 152/865 [05:12<28:02,  2.36s/it]

...

Text Preprocessing:  86%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@         | 741/865 [23:58<02:39,  1.28s/it]

오류!


Text Preprocessing:  97%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  | 842/865 [27:05<00:36,  1.58s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 865/865 [27:44<00:00,  1.92s/it]


In [26]:
MBTI['ENTP'].to_csv('MBTI_ENTP_after.csv', encoding='utf-8', index=False)

## ESFJ

In [27]:
progress_bar = tqdm(total=len(MBTI['ESFJ']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @", colour='green')

MBTI['ESFJ']['pp_content'] = MBTI['ESFJ']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:  13%|@@@@@@@                                                      | 108/858 [03:34<23:04,  1.85s/it]

오류!


Text Preprocessing:  18%|@@@@@@@@@@                                                   | 153/858 [04:57<28:59,  2.47s/it]

오류!


Text Preprocessing:  20%|@@@@@@@@@@@@                                                 | 174/858 [05:28<11:30,  1.01s/it]

...

Text Preprocessing:  97%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@   | 829/858 [25:07<02:00,  4.16s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ | 856/858 [25:54<00:03,  1.52s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 858/858 [26:11<00:00,  1.83s/it]


In [28]:
MBTI['ESFJ'].to_csv('MBTI_ESFJ_after.csv', encoding='utf-8', index=False)

## ESFP

In [29]:
progress_bar = tqdm(total=len(MBTI['ESFP']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @")

MBTI['ESFP']['pp_content'] = MBTI['ESFP']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   6%|@@@                                                           | 51/851 [01:39<23:59,  1.80s/it]

오류!


Text Preprocessing:  11%|@@@@@@                                                        | 96/851 [02:57<21:19,  1.69s/it]

오류!


Text Preprocessing:  12%|@@@@@@@                                                      | 100/851 [03:03<17:42,  1.42s/it]

...

Text Preprocessing:  88%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@        | 747/851 [23:33<05:37,  3.24s/it]

오류!


Text Preprocessing:  93%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@     | 793/851 [25:05<01:09,  1.20s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 851/851 [26:50<00:00,  1.89s/it]


In [30]:
MBTI['ESFP'].to_csv('MBTI_ESFP_after.csv', encoding='utf-8', index=False)

## ESTJ

In [15]:
progress_bar = tqdm(total=len(MBTI['ESTJ']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @")

MBTI['ESTJ']['pp_content'] = MBTI['ESTJ']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   3%|@                                                             | 24/863 [00:51<22:39,  1.62s/it]

오류!


Text Preprocessing:   3%|@                                                             | 25/863 [00:53<23:55,  1.71s/it]

오류!


Text Preprocessing:  10%|@@@@@@                                                        | 87/863 [02:45<26:49,  2.07s/it]

...

Text Preprocessing:  99%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ | 851/863 [23:20<00:18,  1.54s/it]

오류!


Text Preprocessing:  99%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ | 857/863 [23:26<00:06,  1.15s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 863/863 [24:13<00:00,  1.68s/it]


In [16]:
MBTI['ESTJ'].to_csv('MBTI_ESTJ_after.csv', encoding='utf-8', index=False)

## ESTP

In [17]:
progress_bar = tqdm(total=len(MBTI['ESTP']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @")

MBTI['ESTP']['pp_content'] = MBTI['ESTP']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   3%|@                                                             | 22/837 [00:44<20:26,  1.51s/it]

오류!


Text Preprocessing:   5%|@@@                                                           | 43/837 [01:21<22:47,  1.72s/it]

오류!


Text Preprocessing:   6%|@@@                                                           | 51/837 [01:34<20:38,  1.58s/it]

...

Text Preprocessing:  83%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@           | 695/837 [19:33<04:16,  1.81s/it]

오류!


Text Preprocessing:  92%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@     | 771/837 [21:39<02:37,  2.38s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 837/837 [23:16<00:00,  1.67s/it]


In [18]:
MBTI['ESTP'].to_csv('MBTI_ESTP_after.csv', encoding='utf-8', index=False)

## INFJ

In [19]:
progress_bar = tqdm(total=len(MBTI['INFJ']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @")

MBTI['INFJ']['pp_content'] = MBTI['INFJ']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   1%|                                                              | 12/857 [00:25<35:44,  2.54s/it]

오류!


Text Preprocessing:   6%|@@@                                                           | 53/857 [01:49<27:27,  2.05s/it]

오류!


Text Preprocessing:   6%|@@@                                                           | 54/857 [01:50<25:48,  1.93s/it]

...

Text Preprocessing:  77%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@               | 659/857 [21:00<03:58,  1.20s/it]

오류!


Text Preprocessing:  99%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ | 852/857 [26:48<00:06,  1.20s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 857/857 [26:55<00:00,  1.89s/it]


In [20]:
MBTI['INFJ'].to_csv('MBTI_INFJ_after.csv', encoding='utf-8', index=False)

## INFP

In [21]:
progress_bar = tqdm(total=len(MBTI['INFP']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @")

MBTI['INFP']['pp_content'] = MBTI['INFP']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   1%|                                                               | 9/863 [00:19<38:06,  2.68s/it]

오류!


Text Preprocessing:   1%|                                                              | 12/863 [00:24<28:35,  2.02s/it]

오류!


Text Preprocessing:   5%|@@                                                            | 39/863 [01:22<31:07,  2.27s/it]

...

Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ | 859/863 [25:46<00:07,  1.96s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ | 861/863 [25:52<00:05,  2.72s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 863/863 [25:55<00:00,  1.80s/it]


In [22]:
MBTI['INFP'].to_csv('MBTI_INFP_after.csv', encoding='utf-8', index=False)

## INTJ

In [23]:
progress_bar = tqdm(total=len(MBTI['INTJ']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @", colour='blue')

MBTI['INTJ']['pp_content'] = MBTI['INTJ']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   9%|@@@@@                                                         | 80/847 [02:29<21:33,  1.69s/it]

오류!


Text Preprocessing:  20%|@@@@@@@@@@@@                                                 | 168/847 [04:58<25:24,  2.25s/it]

오류!


Text Preprocessing:  32%|@@@@@@@@@@@@@@@@@@@                                          | 275/847 [08:02<18:10,  1.91s/it]

...

Text Preprocessing:  92%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@     | 782/847 [21:55<01:37,  1.50s/it]

오류!


Text Preprocessing:  99%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ | 842/847 [23:38<00:09,  1.81s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 847/847 [23:49<00:00,  1.69s/it]


In [24]:
MBTI['INTJ'].to_csv('MBTI_INTJ_after.csv', encoding='utf-8', index=False)

## INTP

In [25]:
progress_bar = tqdm(total=len(MBTI['INTP']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @", colour='purple')

MBTI['INTP']['pp_content'] = MBTI['INTP']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:  14%|@@@@@@@@                                                     | 117/830 [03:25<20:12,  1.70s/it]

오류!


Text Preprocessing:  34%|@@@@@@@@@@@@@@@@@@@@                                         | 284/830 [08:11<16:10,  1.78s/it]

오류!


Text Preprocessing:  46%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@                                 | 384/830 [11:28<16:17,  2.19s/it]

...

Text Preprocessing:  90%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@      | 749/830 [22:46<03:33,  2.64s/it]

오류!


Text Preprocessing:  95%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    | 788/830 [24:08<01:39,  2.38s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 830/830 [25:18<00:00,  1.83s/it]


In [26]:
MBTI['INTP'].to_csv('MBTI_INTP_after.csv', encoding='utf-8', index=False)

## ISFJ

In [27]:
progress_bar = tqdm(total=len(MBTI['ISFJ']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @", colour='green')

MBTI['ISFJ']['pp_content'] = MBTI['ISFJ']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:  11%|@@@@@@                                                        | 94/867 [02:45<09:32,  1.35it/s]

오류!


Text Preprocessing:  20%|@@@@@@@@@@@                                                  | 170/867 [04:53<22:17,  1.92s/it]

오류!


Text Preprocessing:  21%|@@@@@@@@@@@@                                                 | 182/867 [05:21<24:38,  2.16s/it]

...

Text Preprocessing:  77%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@               | 666/867 [18:35<06:53,  2.06s/it]

오류!


Text Preprocessing:  79%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@             | 686/867 [19:04<03:53,  1.29s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 867/867 [24:15<00:00,  1.68s/it]


In [28]:
MBTI['ISFJ'].to_csv('MBTI_ISFJ_after.csv', encoding='utf-8', index=False)

## ISFP

In [29]:
progress_bar = tqdm(total=len(MBTI['ISFP']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @")

MBTI['ISFP']['pp_content'] = MBTI['ISFP']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   0%|                                                               | 1/873 [00:02<41:12,  2.84s/it]

오류!


Text Preprocessing:   3%|@                                                             | 28/873 [00:54<23:40,  1.68s/it]

오류!


Text Preprocessing:  14%|@@@@@@@@                                                     | 119/873 [03:22<27:51,  2.22s/it]

...

Text Preprocessing:  97%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  | 846/873 [23:18<00:33,  1.23s/it]

오류!


Text Preprocessing:  99%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ | 866/873 [23:44<00:04,  1.55it/s]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 873/873 [23:55<00:00,  1.64s/it]


In [30]:
MBTI['ISFP'].to_csv('MBTI_ISFP_after.csv', encoding='utf-8', index=False)

## ISTJ

In [36]:
progress_bar = tqdm(total=len(MBTI['ISTJ']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @")

MBTI['ISTJ']['pp_content'] = MBTI['ISTJ']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:  13%|@@@@@@@@                                                     | 115/854 [03:46<20:39,  1.68s/it]

오류!


Text Preprocessing:  19%|@@@@@@@@@@@                                                  | 166/854 [05:36<25:58,  2.27s/it]

오류!


Text Preprocessing:  29%|@@@@@@@@@@@@@@@@@                                            | 247/854 [07:57<13:15,  1.31s/it]

...

Text Preprocessing:  82%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@            | 698/854 [21:35<02:41,  1.04s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ | 850/854 [26:31<00:13,  3.35s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 854/854 [26:35<00:00,  1.87s/it]


In [37]:
MBTI['ISTJ'].to_csv('MBTI_ISTJ_after.csv', encoding='utf-8', index=False)

## ISTP

In [38]:
progress_bar = tqdm(total=len(MBTI['ISTP']['content']),
                   desc= 'Text Preprocessing',
                   ncols=120, ascii=" @")

MBTI['ISTP']['pp_content'] = MBTI['ISTP']['content'].apply(preprocess_and_update_progress)

progress_bar.close()

Text Preprocessing:   7%|@@@@                                                          | 58/852 [02:10<27:46,  2.10s/it]

오류!


Text Preprocessing:  25%|@@@@@@@@@@@@@@@                                              | 212/852 [06:57<26:07,  2.45s/it]

오류!


Text Preprocessing:  28%|@@@@@@@@@@@@@@@@@                                            | 242/852 [07:49<12:20,  1.21s/it]

...

Text Preprocessing:  64%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@                       | 543/852 [17:33<10:53,  2.11s/it]

오류!


Text Preprocessing:  83%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@           | 703/852 [24:32<06:40,  2.69s/it]

오류!


Text Preprocessing: 100%|@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@| 852/852 [29:53<00:00,  2.11s/it]


In [39]:
MBTI['ISTP'].to_csv('MBTI_ISTP_after.csv', encoding='utf-8', index=False)

* * *
# Text Preprocessing 2
데이터 stopword 추가 처리 및 결측치 제거

In [3]:
ENFJ = pd.read_csv('MBTI_ENFJ_after.csv', encoding='utf-8')
ENFP = pd.read_csv('MBTI_ENFP_after.csv', encoding='utf-8')
ENTJ = pd.read_csv('MBTI_ENTJ_after.csv', encoding='utf-8')
ENTP = pd.read_csv('MBTI_ENTP_after.csv', encoding='utf-8')
ESFJ = pd.read_csv('MBTI_ESFJ_after.csv', encoding='utf-8')
ESFP = pd.read_csv('MBTI_ESFP_after.csv', encoding='utf-8')
ESTJ = pd.read_csv('MBTI_ESTJ_after.csv', encoding='utf-8')
ESTP = pd.read_csv('MBTI_ESTP_after.csv', encoding='utf-8')
INFJ = pd.read_csv('MBTI_INFJ_after.csv', encoding='utf-8')
INFP = pd.read_csv('MBTI_INFP_after.csv', encoding='utf-8')
INTJ = pd.read_csv('MBTI_INTJ_after.csv', encoding='utf-8')
INTP = pd.read_csv('MBTI_INTP_after.csv', encoding='utf-8')
ISFJ = pd.read_csv('MBTI_ISFJ_after.csv', encoding='utf-8')
ISFP = pd.read_csv('MBTI_ISFP_after.csv', encoding='utf-8')
ISTJ = pd.read_csv('MBTI_ISTJ_after.csv', encoding='utf-8')
ISTP = pd.read_csv('MBTI_ISTP_after.csv', encoding='utf-8')

In [4]:
MBTI = {'ENFJ': ENFJ, 'ENFP': ENFP, 'ENTJ': ENFJ, 'ENTP': ENFP, 'ESFJ': ESFJ, 'ESFP': ESFP, 'ESTJ': ESTJ, 'ESTP': ESTP, 'INFJ': INFJ, 'INFP': INFP, 'INTJ': INTJ, 'INTP': INTP, 'ISFJ': ISFJ, 'ISFP': ISFP, 'ISTJ': ISTJ, 'ISTP': ISTP}

In [5]:
for m in MBTI: # 결측치 제거 및 인덱스 초기화
    MBTI.get(m).dropna(inplace=True)
    MBTI.get(m).reset_index(inplace=True) # 아래의 오류는 이미 reset_index가 완료된 상태라서 나는 오류입니다!
    print("{} 결측치 행 제거 완료".format(m))

ValueError: cannot insert level_0, already exists

In [6]:
with open('./mbti_stop.txt', 'r', encoding='utf-8') as f:
    stopwords2 = f.readlines()
stopwords2 = [x.replace('\n','') for x in stopwords2]

In [7]:
def goodbye_stopword(text):
    tokens = text.split() # 어차피 이미 토큰화된 상태이므로 그냥 띄어쓰기 단위로 자르기

    result = [token for token in tokens if token not in stopwords2]
    result = ' '.join(result)
    
    return result

In [8]:
for m in MBTI:
    MBTI.get(m)['pp_content'] = MBTI.get(m)['pp_content'].apply(goodbye_stopword)
    MBTI.get(m).to_csv('MBTI_{}_after.csv'.format(m), encoding='utf-8', index=False)
    print("{} 완료!".format(m))

ENFJ 완료!
ENFP 완료!
ENTJ 완료!
ENTP 완료!
ESFJ 완료!
ESFP 완료!
ESTJ 완료!
ESTP 완료!
INFJ 완료!
INFP 완료!
INTJ 완료!
INTP 완료!
ISFJ 완료!
ISFP 완료!
ISTJ 완료!
ISTP 완료!
